# Polymer Brush Simulation Analysis Notebook

In [ ]:
%matplotlib widget
import ipywidgets as widgets
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
from matplotlib import rcParams
import math
import sys
import pickle
import warnings
warnings.filterwarnings('ignore')
import logging
logging.getLogger().setLevel(logging.CRITICAL)
import pb_sims as pb

In [3]:
os.getcwd()

'/home/deck/Desktop/code/ma-abdelbar/sci-md-polymer-brush-bilayers/Analysis'

In [ ]:
series_paths = [r'..\Loop\Sims',
                r'..\Linear\Sims',
                r'..\Loop\Sims',
                r'..\BB\Sims']

Topologies = []
i = 0
for series_path in series_paths:
    if os.path.isdir(series_path):
        Topology1 = pb.Sims(series_path)
        Topologies.append(Topology1)
    else:
        print('The {0} directory is missing, the widgets will load the data from the pickled files.'.format(series_path))

    i += 1


The ..\Cyclic\Sims directory is missing, the widgets will load the data from the pickled files.
The ..\Linear\Sims directory is missing, the widgets will load the data from the pickled files.
The ..\Loop\Sims directory is missing, the widgets will load the data from the pickled files.
The ..\BB\Sims directory is missing, the widgets will load the data from the pickled files.


In [3]:
if os.path.isdir('data') == False:
    os.mkdir('data')
T_info_f = 'data/T_info'
if (os.path.isfile(T_info_f) == True):
    with open(T_info_f,'rb') as f:
        unpickler = pickle.Unpickler(f)
        T_info = unpickler.load()
else:
    T_info = []
    for Topology in Topologies:
        T_info.append(Topology.info())
    with open(T_info_f,'wb') as f:
            pickle.dump(T_info,f)

T_Names = [T_info[i][0].split()[0] for i in range(len(T_info))]
T_Ndirs = T_info[0][1]
T_Mdirs = T_info[0][2]
T_Xdirs = T_info[0][3]
T_PDdirs = T_info[0][4]
T_Vdirs = T_info[0][5]


In [4]:
tsteps = 40
Lx = 30
Ly = 25
areaLJ = Lx *Ly
T_Gdirs = [r'$\rho_g ={0:.2f}$'.format(int(M[2:])/areaLJ) for M in T_Mdirs]
print(T_Gdirs)

['$\\rho_g =0.02$', '$\\rho_g =0.05$', '$\\rho_g =0.06$', '$\\rho_g =0.08$', '$\\rho_g =0.13$', '$\\rho_g =0.21$', '$\\rho_g =0.28$']


In [5]:
print('Brush Topologies and Parameter Values')
print('--------------------------------------')
for i in T_info:
    for j in i:
        print(j, end = '\n')
    print('',sep='\n')

Brush Topologies and Parameter Values
--------------------------------------
Cyclic Brushes
['N=30', 'N=40', 'N=50', 'N=60']
['M=18', 'M=38', 'M=44', 'M=58', 'M=100', 'M=158', 'M=208']
['X=0']
['D=10', 'D=20', 'D=30', 'D=40', 'D=50', 'D=70']
['V=0.0010000000000000000208', 'V=0.0030000000000000000625', 'V=0.010000000000000000208', 'V=0.02999999999999999889', 'V=0.10000000000000000555', 'V=0.2999999999999999889', 'V=1']

Linear Brushes
['N=30', 'N=40', 'N=50', 'N=60']
['M=18', 'M=38', 'M=44', 'M=58', 'M=100', 'M=158', 'M=208']
['X=0']
['D=10', 'D=20', 'D=30', 'D=40', 'D=50', 'D=70']
['V=0.0010000000000000000208', 'V=0.0030000000000000000625', 'V=0.010000000000000000208', 'V=0.02999999999999999889', 'V=0.10000000000000000555', 'V=0.2999999999999999889', 'V=1']

Loop Brushes
['N=30', 'N=40', 'N=50', 'N=60']
['M=18', 'M=38', 'M=44', 'M=58', 'M=100', 'M=158', 'M=208']
['X=0']
['D=10', 'D=20', 'D=30', 'D=40', 'D=50', 'D=70']
['V=0.0010000000000000000208', 'V=0.0030000000000000000625', 'V=0.01

In [6]:
# Read the equilibrium data
eDPS_f = 'data/eDPS_data'
RhovH_f = 'data/RhovH_data'
eDPSs_f = 'data/eDPSs_data'

if (os.path.isfile(eDPS_f) == True and os.path.isfile(RhovH_f) == True):
    with open(eDPS_f,'rb') as f:
        unpickler = pickle.Unpickler(f)
        eDPS = unpickler.load()
    with open(RhovH_f,'rb') as f:
        unpickler = pickle.Unpickler(f)
        RhovH = unpickler.load()
else:
    eDPS = []
    eDPSs = []
    RhovH = []
    for Topology in Topologies:
            eDPSd = Topology.profiles(['bbeads_edz'], PDis=[0], Vis=[0], Z2D=False)
            eDPSr = pb.reorder_data(eDPSd)
            eDPSsd = Topology.profiles(['bbeads_edz','sbeads_edz'], PDis=[0], Vis=[0], Z2D=False)
            eDPS.append(eDPSr[:,:,:,:,:,:,:])
            eDPSs.append(eDPSsd[:,:,:,:,:,:,:])
            Hsr = Topology.brush_heights(eDPSd)
            Rhosd = Topology.simVars(svar='surfcov', PDis=[0], Vis=[0])
            if Topology.topology in "Cyclic Brushes":
                Rhosd = Rhosd * 2.0
            RhovHr = np.ma.concatenate((Rhosd, Hsr), axis=-2)
            RhovH.append(RhovHr[:, :, :, :, :, :, :])
    with open(eDPS_f,'wb') as f:
        pickle.dump(eDPS,f)
    with open(RhovH_f,'wb') as f:
        pickle.dump(RhovH,f)
    with open(eDPSs_f,'wb') as f:
        pickle.dump(eDPSs,f)
        
        

In [7]:
cDPS_f = 'data/cDPS_data'
if os.path.isfile(cDPS_f):
    with open(cDPS_f,'rb') as f:
        unpickler = pickle.Unpickler(f)
        cDPS = unpickler.load()
else:
    cDPS = []
    for Topology in Topologies:
        cDPSd = Topology.profiles(['bbeads_cdz', 'tbeads_cdz','sbeads_cdz', 'abeads_cdz'], Vis=[0],Z2D=True)
        cDPS.append(cDPSd[:, :, :, :, 0, :, :])
    with open(cDPS_f,'wb') as f:
        pickle.dump(cDPS,f)


In [8]:
PvD_f = 'data/PvD_data'
if os.path.isfile(PvD_f):
    with open(PvD_f,'rb') as f:
        unpickler = pickle.Unpickler(f)
        PvD = unpickler.load()
else:
    PvD = []
    for Topology in Topologies:
        PvDd = Topology.thermo(2, ['f_D', 'f_pzz'],steps=tsteps, Vis=[0], avg=True)
        PvD.append(PvDd[:, :, :, :, 0, :, :])
    with open(PvD_f,'wb') as f:
        pickle.dump(PvD,f)
    


In [9]:
OvD_f = 'data/OvD_data'
NMFHvD_f = 'data/NMFHvD_data'
FMFHvD_f = 'data/FMFHvD_data'
MDvD_f = 'data/MDvD_data'
if os.path.isfile(OvD_f):
    with open(OvD_f,'rb') as f:
        unpickler = pickle.Unpickler(f)
        OvD = unpickler.load()
if os.path.isfile(NMFHvD_f):
    with open(NMFHvD_f,'rb') as f:
        unpickler = pickle.Unpickler(f)
        NMFHvD = unpickler.load()
if os.path.isfile(FMFHvD_f):
    with open(FMFHvD_f,'rb') as f:
        unpickler = pickle.Unpickler(f)
        FMFHvD = unpickler.load()
if os.path.isfile(MDvD_f):
    with open(MDvD_f,'rb') as f:
        unpickler = pickle.Unpickler(f)
        MDvD = unpickler.load()
else:
    OvD = []
    NMFHvD = []
    FMFHvD = []
    MDvD = []
    
    for Topology in Topologies:
        iDPSd = Topology.profiles(['bbeads_cdz', 'tbeads_cdz'], Vis=[0], Z2D=False)
        Osd = Topology.overlap(iDPSd)
        NMFHd = Topology.NMFHS(iDPSd)
        FMFHd = Topology.FMFHS(iDPSd)
        MDd = Topology.mid_den(iDPSd)
        Dsd = Topology.thermo(2, tvars=['f_D'],steps=tsteps, Vis=[0])
        OvDr = np.ma.concatenate((Dsd, Osd), axis=-2)
        OvD.append(OvDr[:, :, :, :, :, :, :])
        NMFHvDr = np.ma.concatenate((Dsd, NMFHd), axis=-2)
        NMFHvD.append(NMFHvDr[:, :, :, :, :, :, :])
        FMFHvDr = np.ma.concatenate((Dsd, FMFHd), axis=-2)
        FMFHvD.append(FMFHvDr[:, :, :, :, :, :, :])
        MDvDr = np.ma.concatenate((Dsd, MDd), axis=-2)
        MDvD.append(MDvDr[:, :, :, :, :, :, :])
    with open(OvD_f,'wb') as f:
        pickle.dump(OvD,f)
    with open(NMFHvD_f,'wb') as f:
        pickle.dump(NMFHvD,f)
    with open(FMFHvD_f,'wb') as f:
        pickle.dump(FMFHvD,f)
    with open(MDvD_f,'wb') as f:
        pickle.dump(MDvD,f)
        

In [10]:
OvDs_f = 'data/OvDs_data'
NMFHvDs_f = 'data/NMFHvDs_data'
FMFHvDs_f = 'data/FMFHvDs_data'
MDvDs_f = 'data/MDvDs_data'
OvV_f = 'data/OvV_data'
NMFHvV_f = 'data/NMFHvV_data'
FMFHvV_f = 'data/FMFHvV_data'
MDvV_f = 'data/MDvV_data'

if os.path.isfile(OvV_f):
    with open(OvDs_f,'rb') as f:
        unpickler = pickle.Unpickler(f)
        OvDs = unpickler.load()
    with open(OvV_f,'rb') as f:
        unpickler = pickle.Unpickler(f)
        OvV = unpickler.load()
    with open(NMFHvDs_f,'rb') as f:
        unpickler = pickle.Unpickler(f)
        NMFHvDs = unpickler.load()
    with open(FMFHvDs_f,'rb') as f:
        unpickler = pickle.Unpickler(f)
        FMFHvDs = unpickler.load()
    with open(NMFHvV_f,'rb') as f:
        unpickler = pickle.Unpickler(f)
        NMFHvV = unpickler.load()
    with open(FMFHvV_f,'rb') as f:
        unpickler = pickle.Unpickler(f)
        FMFHvV = unpickler.load()
    with open(MDvDs_f,'rb') as f:
        unpickler = pickle.Unpickler(f)
        MDvDs = unpickler.load()
    with open(MDvV_f,'rb') as f:
        unpickler = pickle.Unpickler(f)
        MDvV = unpickler.load()
    
    
else:
    OvDs = []
    OvV = []
    NMFHvDs = []
    FMFHvDs = []
    NMFHvV = []
    FMFHvV = []
    MDvDs = []
    MDvV = []
    for Topology in Topologies:
        iDPSds = Topology.profiles(['bbeads_sdz', 'tbeads_sdz'], Z2D=False)
        Osds = Topology.overlap(iDPSds)
        NMFHds = Topology.NMFHS(iDPSds)
        FMFHds = Topology.FMFHS(iDPSds)
        MDds = Topology.mid_den(iDPSds)
        Dsds = Topology.thermo(3, tvars=['f_D'],steps=tsteps)
        OvDr = np.ma.concatenate((Dsds, Osds), axis=-2)
        OvDs.append(OvDr[:, :, :, :, :, :, :])
        NMFHvDr = np.ma.concatenate((Dsds, NMFHds), axis=-2)
        NMFHvDs.append(NMFHvDr[:, :, :, :, :, :, :])
        FMFHvDr = np.ma.concatenate((Dsds, FMFHds), axis=-2)
        FMFHvDs.append(FMFHvDr[:, :, :, :, :, :, :])
        MDvDr = np.ma.concatenate((Dsds, MDds), axis=-2)
        MDvDs.append(MDvDr[:, :, :, :, :, :, :])
        Vsds= Topology.simVars('Vwall')
        OvVr = np.ma.concatenate((Vsds, Osds), axis=-2)
        OvV.append(OvVr[:, :, :, :, :, :, :])
        NMFHvVr = np.ma.concatenate((Vsds, NMFHds), axis=-2)
        NMFHvV.append(NMFHvVr[:, :, :, :, :, :, :])
        FMFHvVr = np.ma.concatenate((Vsds, FMFHds), axis=-2)
        FMFHvV.append(FMFHvVr[:, :, :, :, :, :, :])
        MDvVr = np.ma.concatenate((Vsds, MDds), axis=-2)
        MDvV.append(MDvVr[:, :, :, :, :, :, :])
    with open(OvDs_f,'wb') as f:
        pickle.dump(OvDs,f)
    with open(OvV_f,'wb') as f:
        pickle.dump(OvV,f)
    with open(NMFHvDs_f,'wb') as f:
        pickle.dump(NMFHvDs,f)
    with open(FMFHvDs_f,'wb') as f:
        pickle.dump(FMFHvDs,f)
    with open(NMFHvV_f,'wb') as f:
        pickle.dump(NMFHvV,f)
    with open(FMFHvV_f,'wb') as f:
        pickle.dump(FMFHvV,f)
    with open(MDvDs_f,'wb') as f:
        pickle.dump(MDvDs,f)
    with open(MDvV_f,'wb') as f:
        pickle.dump(MDvV,f)

In [11]:
# Read shear temperature-density-velocity profiles
TDV_f = 'data/TDV_data'
if os.path.isfile(TDV_f):
    with open(TDV_f,'rb') as f:
        unpickler = pickle.Unpickler(f)
        TDV = unpickler.load()
else:
    TDV = []
    for Topology in Topologies:
        print(Topology.info())
        TDVd = Topology.profiles(['temp_sz', 'abeads_sdz', 'velp_sz'], Z2D=True)
        TDV.append(TDVd[:, :, :, :, :, :, :])
    with open(TDV_f,'wb') as f:
        pickle.dump(TDV,f)

['Cyclic Brushes', ['N=30', 'N=40', 'N=50', 'N=60'], ['M=18', 'M=38', 'M=44', 'M=58', 'M=100', 'M=158', 'M=208'], ['X=0'], ['D=10', 'D=20', 'D=30', 'D=40', 'D=50', 'D=70'], ['V=0.0010000000000000000208', 'V=0.0030000000000000000625', 'V=0.010000000000000000208', 'V=0.02999999999999999889', 'V=0.10000000000000000555', 'V=0.2999999999999999889', 'V=1']]
['Linear Brushes', ['N=30', 'N=40', 'N=50', 'N=60'], ['M=18', 'M=38', 'M=44', 'M=58', 'M=100', 'M=158', 'M=208'], ['X=0'], ['D=10', 'D=20', 'D=30', 'D=40', 'D=50', 'D=70'], ['V=0.0010000000000000000208', 'V=0.0030000000000000000625', 'V=0.010000000000000000208', 'V=0.02999999999999999889', 'V=0.10000000000000000555', 'V=0.2999999999999999889', 'V=1']]
['Loop Brushes', ['N=30', 'N=40', 'N=50', 'N=60'], ['M=18', 'M=38', 'M=44', 'M=58', 'M=100', 'M=158', 'M=208'], ['X=0'], ['D=10', 'D=20', 'D=30', 'D=40', 'D=50', 'D=70'], ['V=0.0010000000000000000208', 'V=0.0030000000000000000625', 'V=0.010000000000000000208', 'V=0.02999999999999999889', 'V=

In [12]:
# Read shear density profiles
sDPS_f = 'data/sDPS_data'
if os.path.isfile(sDPS_f):
    with open(sDPS_f,'rb') as f:
        unpickler = pickle.Unpickler(f)
        sDPS = unpickler.load()
else:
    sDPS = []
    for Topology in Topologies:
        sDPSd = Topology.profiles(['bbeads_sdz', 'tbeads_sdz','sbeads_sdz', 'abeads_sdz'],Z2D=True)
        sDPS.append(sDPSd[:, :, :, :, :, :, :])
    with open(sDPS_f,'wb') as f:
        pickle.dump(sDPS,f)

In [13]:

# Read shear force and configurational data

RvS_f = 'data/RvS_data'
RvW_f = 'data/RvW_data'
PvS_f = 'data/PvS_data'
FvS_f = 'data/FvS_data'
FvS_m_f = 'data/FvS_m_data'
FvW_f = 'data/FvW_data'
FvW_m_f = 'data/FvW_m_data'
FvM = 'data/FvM_data'
VvS_f = 'data/VvS_data'
VvS_m_f = 'data/VvS_m_data'
VvW_f = 'data/VvW_data'
VvW_m_f = 'data/VvW_m_data'
FvM = 'data/FvM_data'
PxvPz_f = 'data/PxvPz_data'
NvM_f = 'data/NvM_data'
Dv_f = 'data/Dv_data'


if os.path.isfile(RvS_f) and os.path.isfile(Dv_f):
    with open(RvS_f,'rb') as f:
        unpickler = pickle.Unpickler(f)
        RvS = unpickler.load()
    with open(RvW_f,'rb') as f:
        unpickler = pickle.Unpickler(f)
        RvW = unpickler.load()
    with open(PvS_f,'rb') as f:
        unpickler = pickle.Unpickler(f)
        PvS = unpickler.load()
    with open(FvS_f,'rb') as f:
        unpickler = pickle.Unpickler(f)
        FvS = unpickler.load()
    with open(FvS_m_f,'rb') as f:
        unpickler = pickle.Unpickler(f)
        FvS_m = unpickler.load()
    with open(FvW_f,'rb') as f:
        unpickler = pickle.Unpickler(f)
        FvW = unpickler.load()
    with open(FvW_m_f,'rb') as f:
        unpickler = pickle.Unpickler(f)
        FvW_m = unpickler.load()
    with open(VvS_f,'rb') as f:
        unpickler = pickle.Unpickler(f)
        VvS = unpickler.load()
    with open(VvS_m_f,'rb') as f:
        unpickler = pickle.Unpickler(f)
        VvS_m = unpickler.load()
    with open(VvW_f,'rb') as f:
        unpickler = pickle.Unpickler(f)
        VvW = unpickler.load()
    with open(VvW_m_f,'rb') as f:
        unpickler = pickle.Unpickler(f)
        VvW_m = unpickler.load()
    with open(PxvPz_f,'rb') as f:
        unpickler = pickle.Unpickler(f)
        PxvPz = unpickler.load()
    with open(NvM_f,'rb') as f:
        unpickler = pickle.Unpickler(f)
        NvM = unpickler.load()
    with open(Dv_f,'rb') as f:
        unpickler = pickle.Unpickler(f)
        Dv = unpickler.load()
        
else:
    RvS = []
    RvW = []
    PvS = []
    FvS = []
    FvS_m = []
    FvW = []
    FvW_m = []
    VvS = []
    VvS_m = []
    VvW = []
    VvW_m = []
    PxvPz = []
    NvM = []
    Dv = []
    for Topology in Topologies:
        RvS_c = Topology.thermo(3, ['f_srate', 'f_Rgx'],steps=tsteps, avg=True)
        Rg2xvS_c = Topology.Rg2x(RvS_c)
        RvS.append(Rg2xvS_c[:, :, :, :, :, :, :])
        src_c = Topology.srate_crit(Rg2xvS_c)
        Rg2xvW_c = Topology.Weissenberg(Rg2xvS_c,src_c)
        RvW.append(Rg2xvW_c[:, :, :, :, :, :, :])
        PvS_c = Topology.thermo(3, ['f_srate', 'f_pxz', 'f_pzz'],steps=tsteps,avg=True)
        PvS.append(PvS_c[:, :, :, :, :, :, :])
        
        FvS_c = Topology.COF(PvS_c)
        FvS.append(FvS_c[:, :, :, :, :, :, :])
        FvS_m_c = Topology.mask(FvS_c, src_c)
        FvS_m.append(FvS_m_c[:, :, :, :, :, :, :])
        FvW.append(Topology.Weissenberg(FvS_c, src_c))
        FvW_m.append(Topology.Weissenberg(FvS_m_c, src_c))
        
        VvS_c = Topology.effVisc(PvS_c)
        VvS.append(VvS_c[:, :, :, :, :, :, :])
        VvS_m_c = Topology.mask(VvS_c, src_c)
        VvS_m.append(VvS_m_c[:, :, :, :, :, :, :])
        VvW.append(Topology.Weissenberg(VvS_c, src_c))
        VvW_m.append(Topology.Weissenberg(VvS_m_c, src_c))
        
        
        
        PxvPzd = Topology.thermo(3, ['f_pzz','f_pxz'],steps=tsteps,avg=True)
        PxvPzr = np.moveaxis(PxvPzd, [0,1,2,3,4,5,6],[0,1,2,4,3,5,6])
        PxvPzr2 = np.moveaxis(PxvPzr,[0,1,2,3,4,5,6],[0,3,2,1,4,5,6])
        PxvPz.append(PxvPzr2[:,:,:,:,:,:,:])
        NvMd = Topology.thermoNM(3)
        NvM.append(NvMd)
        Dvd =  Topology.thermo(3, ['f_D'],steps=tsteps,avg=True)
        Dv.append(Dvd[:,:,:,:,:,:,:])
    with open(RvS_f,'wb') as f:
        pickle.dump(RvS,f)
    with open(RvW_f,'wb') as f:
        pickle.dump(RvW,f)
    with open(PvS_f,'wb') as f:
        pickle.dump(PvS,f)
    with open(FvS_f,'wb') as f:
        pickle.dump(FvS,f)
    with open(FvS_m_f,'wb') as f:
        pickle.dump(FvS_m,f)
    with open(FvW_f,'wb') as f:
        pickle.dump(FvW,f)
    with open(FvW_m_f,'wb') as f:
        pickle.dump(FvW_m,f)
    with open(VvS_f,'wb') as f:
        pickle.dump(VvS,f)
    with open(VvS_m_f,'wb') as f:
        pickle.dump(VvS_m,f)
    with open(VvW_f,'wb') as f:
        pickle.dump(VvW,f)
    with open(VvW_m_f,'wb') as f:
        pickle.dump(VvW_m,f)
    with open(PxvPz_f,'wb') as f:
        pickle.dump(PxvPz,f)
    with open(NvM_f,'wb') as f:
        pickle.dump(NvM,f)
    with open(Dv_f,'wb') as f:
        pickle.dump(Dv,f)
    

In [14]:
# Prepare data for PsvS plot
PvS_c = [PvS[i] for i in range(len(PvS))]
PsvS = []
for i in range(len(PvS_c)):
    Psv = np.ma.expand_dims(PvS_c[i][:,:,:,:,:,0,:],axis=-2)
    Sv = np.ma.expand_dims(PvS_c[i][:,:,:,:,:,1,:],axis=-2)
    
    PsvS.append(np.ma.concatenate((Psv,Sv),axis=-2))

In [15]:
RvV_f = 'data/RvV_data'
PvV_f = 'data/PvV_data'
FvV_f = 'data/FvV_data'
VvV_f = 'data/VvV_data'

if os.path.isfile(RvV_f) and os.path.isfile(PvV_f):
    with open(RvV_f,'rb') as f:
        unpickler = pickle.Unpickler(f)
        RvV = unpickler.load()
    with open(PvV_f,'rb') as f:
        unpickler = pickle.Unpickler(f)
        PvV = unpickler.load()
    with open(FvV_f,'rb') as f:
        unpickler = pickle.Unpickler(f)
        FvV = unpickler.load()
    with open(VvV_f,'rb') as f:
        unpickler = pickle.Unpickler(f)
        VvV = unpickler.load()
else:
    RvV = []
    PvV = []
    FvV = []
    VvV = []
    for Topology in Topologies:
        Vv = Topology.simVars('Vwall')
        Rv = Topology.thermo(3, ['f_Rgx'],steps=tsteps, avg=True)
        RvV_c = np.ma.concatenate((Vv,Rv),axis=-2)
        Rg2xvV_c = Topology.Rg2x(RvV_c)
        RvV.append(Rg2xvV_c[:, :, :, :, :, :, :])
        
        Pxz = Topology.thermo(3, [ 'f_pxz',],steps=tsteps,avg=True)
        Pzz = Topology.thermo(3, [ 'f_pzz',],steps=tsteps,avg=True)
        
        PvV_c = np.ma.concatenate((Vv,Pxz,Vv,Pzz),axis=-2)
        PvV.append(PvV_c[:, :, :, :, :, :, :])
        
        FvV_c = Topology.COF(PvV_c)
        FvV.append(FvV_c[:, :, :, :, :, :, :])
        
        VvV_c = Topology.effVisc(PvV_c)
        VvV.append(VvV_c[:, :, :, :, :, :, :])
    with open(RvV_f,'wb') as f:
        pickle.dump(RvV,f)  
    with open(PvV_f,'wb') as f:
        pickle.dump(PvV,f)
    with open(FvV_f,'wb') as f:
        pickle.dump(FvV,f)
    with open(VvV_f,'wb') as f:
        pickle.dump(VvV,f)
    


###### 